In [1]:
%load_ext autoreload
%autoreload 2
from datasetUtil import *
import json
import pickle
import matplotlib.pyplot as plt

In [2]:
one_billion_path = "1-billion-word-language-modeling-benchmark-r13output/heldout-monolingual.tokenized.shuffled"

In [3]:
one_billion_sentence_tokens = readOneBillion(one_billion_path)

1-billion-word-language-modeling-benchmark-r13output/heldout-monolingual.tokenized.shuffled/news.en.heldout-00000-of-00050
1-billion-word-language-modeling-benchmark-r13output/heldout-monolingual.tokenized.shuffled/news.en.heldout-00001-of-00050
1-billion-word-language-modeling-benchmark-r13output/heldout-monolingual.tokenized.shuffled/news.en.heldout-00002-of-00050
1-billion-word-language-modeling-benchmark-r13output/heldout-monolingual.tokenized.shuffled/news.en.heldout-00003-of-00050
1-billion-word-language-modeling-benchmark-r13output/heldout-monolingual.tokenized.shuffled/news.en.heldout-00004-of-00050
1-billion-word-language-modeling-benchmark-r13output/heldout-monolingual.tokenized.shuffled/news.en.heldout-00005-of-00050
1-billion-word-language-modeling-benchmark-r13output/heldout-monolingual.tokenized.shuffled/news.en.heldout-00006-of-00050
1-billion-word-language-modeling-benchmark-r13output/heldout-monolingual.tokenized.shuffled/news.en.heldout-00007-of-00050
1-billion-word-l

In [4]:
one_billion_sentence_tokens = adaptERNIEtokenization(one_billion_sentence_tokens)

  4%|▍         | 12602/306688 [00:00<00:02, 126019.82it/s]

Parsed to ERNIE tokens!


100%|██████████| 306688/306688 [00:02<00:00, 104597.91it/s]


In [6]:
mergedCounts = countCorpus(one_billion_sentence_tokens)

100%|██████████| 306688/306688 [00:04<00:00, 74530.35it/s]


In [ ]:
#char_path = "charVocab.txt"
#word_path = "wordVocab.txt"
#charList, wordList = readVocab(char_path, word_path)

In [17]:
frequentWordCount = mergedCounts[:30000]
wordList = [word for word, count in frequentWordCount]
charList = charVocab(mergedCounts)
with open("wordVocab.txt","w") as f:
    for word in wordList:
        f.write(word+"\n")
    f.write("<unk>"+"\n")
    f.write("<oov>"+"\n")
with open("charVocab.txt","w") as f:
    for char in charList:
        f.write(char+"\n")

In [18]:
oov_one_billion_sentence_tokens = filterOutlierSentence(one_billion_sentence_tokens)

In [19]:
len(oov_one_billion_sentence_tokens)

306544

In [20]:
filtered_tokens, _ = excludeSentenceWithUnknownWords(wordList, oov_one_billion_sentence_tokens)

In [21]:
len(filtered_tokens)

191005

In [24]:
N_test = 5000
test_sentences = filtered_tokens[:N_test]
dev_sentences = filtered_tokens[N_test:2*N_test]
train_sentences = filtered_tokens[2*N_test:]

In [26]:
print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))

181005
5000
5000


In [27]:
with open("train1B.txt","w") as f:
    for sentence in train_sentences:
        f.write(" ".join(sentence)+"\n")
with open("dev1B.txt","w") as f:
    for sentence in dev_sentences:
        f.write(" ".join(sentence)+"\n")
with open("test1B.txt","w") as f:
    for sentence in test_sentences:
        f.write(" ".join(sentence)+"\n")

In [28]:
berkbeck = read_berkbeck("missp.dat.txt")

In [29]:
natural_replacement = readYonatan("en.natural.txt")

In [30]:
mergedWordReplacement = mergeWordReplacement(berkbeck, natural_replacement)

In [33]:
#trainWordReplacement = sampleReplacement(mergedWordReplacement, 0.8)
#with open("train_replacement.json","w") as f:
#    json.dump(trainWordReplacement, f)

In [34]:
with open("train_replacement.json") as f:
    trainWordReplacement = json.load(f)

In [35]:
replaced_one_billion_sentence_tokens, gold_labels = wordReplacement(trainWordReplacement, train_sentences, word_vocab = None, sigma=0.2)
saveNoisyInput("train.txt", replaced_one_billion_sentence_tokens)
saveNoisyInputLabel("train_label.jsonl", gold_labels)

181005it [00:05, 32747.55it/s]


In [36]:
replaced_one_billion_sentence_tokens, gold_labels = wordReplacement(trainWordReplacement, dev_sentences, word_vocab = None, sigma=0.2)
saveNoisyInput("dev.txt", replaced_one_billion_sentence_tokens)
saveNoisyInputLabel("dev_label.jsonl", gold_labels)

5000it [00:00, 35036.33it/s]


In [37]:
replaced_one_billion_sentence_tokens, gold_labels = wordReplacement(mergedWordReplacement, test_sentences, word_vocab = None, sigma=0.2)
saveNoisyInput("test.txt", replaced_one_billion_sentence_tokens)
saveNoisyInputLabel("test_label.jsonl", gold_labels)

5000it [00:00, 37385.99it/s]


In [39]:
replaced_one_billion_sentence_tokens, gold_labels = charReplacement(train_sentences, word_vocab = None, sigma=0.2)
saveNoisyInput("char_train.txt", replaced_one_billion_sentence_tokens)
saveNoisyInputLabel("char_train_label.jsonl", gold_labels)

181005it [00:13, 13808.35it/s]
